In [1]:
#==============================================================================================
# 기존 bert vocab 형태의 새로운 vocab tokenizer 만들기
#
# => 기존 bert vocab을 모두 지우고, 새로운 vocab을 가지는 tokenizer 만들기
# => AutoTokenizer.train_new_from_iterator() 이용
# => 기존 bert-base-cased 모델에 있는 기존 영문 vocab들 대신에 한국어 말뭉치를 가지고 새롭게 한국어 vocab을 만든다.
#
# => 말뭉치데이터는 python 제너레이터 형태로 만든다.
# => yield 문을 사용하여 for 루프 내에서 제너레이터(generator)를 정의한다.
#
# 출처 : https://wikidocs.net/166821
#
# 허깅페이스 허브를 이용하여, 토크너나이즈 올리기
# => git 설치되어 있어야 함.
# => 출처 : https://zerolang.tistory.com/65
#
#==============================================================================================
import konlpy
from konlpy.tag import Mecab
from tqdm.notebook import tqdm
from nltk import FreqDist
import numpy as np
import torch
from transformers import BertTokenizer
import os
from os import sys
sys.path.append('../')
from myutils import seed_everything

seed_everything(111)

# 입력 말뭉치
corpus = '../../data11/my_corpus/re-kowiki202206-moco2.txt'
# MECAB으로 전처리된 말뭉치
out_corpus = '../../data11/my_corpus/re-kowiki202206-moco2-mecab.txt'

In [2]:
#한국어 말뭉치인 경우에는 mecab(메케브)로 전처리 해줌
##### Mecab 를 이용하여, 말뭉치 형태소 분석 
with open(corpus, 'r', encoding='utf-8') as f:
    data = f.read().split('\n')

print(data[:3])
print(f'*len:{len(data)}')

mecab = Mecab()
# False: '어릴때' -> '어릴', '때' 로 일반화 시킴
total_morph=[]

for sentence in tqdm(data):
    morph_sentence = mecab.morphs(sentence)
    total_morph.append(morph_sentence)
    
# 파일로 저장
with open(out_corpus, 'w', encoding='utf-8') as f:
    for line in tqdm(total_morph):
        # ** 앞에 ' ' 공백을 넣어서 형태로 별로 분리하여 저장함
        f.write(' '.join(line)+'\n')

['스포츠 팀 문화 활동 해외 여행 등의 다양한 과외활동을 제공한다', '남자 자유형 200m 종목의 예선기준 기록은 1분 47초 02이다', '1785년 정조 9 당시 성균관 대사성 민종현 이 왕명에 의해 편찬한 태학지 건치 에 반궁도 가 목판화로 있는데 본 계첩과 30여 년의 시간적 차이가 있어서 그간에 변화된 성균관의 건물을 비교해 볼 수 있다']
*len:7051665


  0%|          | 0/7051665 [00:00<?, ?it/s]

  0%|          | 0/7051665 [00:00<?, ?it/s]

In [3]:
# wiki_20190620.txt 말뭉치 불러옴.
#corpus = '../../data11/my_corpus/re-moco-corpus2.txt'
#corpus = '../../data11/my_corpus/re-kowiki-202206.txt'
#corpus = '../../data11/my_corpus/test.txt'

with open(out_corpus, 'r', encoding='utf-8') as f:
    data = [line for line in tqdm(f.read().splitlines()) if (len(line) > 0 and not line.isspace())]

print(data[:3])
print(len(data))

  0%|          | 0/7051665 [00:00<?, ?it/s]

['스포츠 팀 문화 활동 해외 여행 등 의 다양 한 과외 활동 을 제공 한다', '남자 자유형 200 m 종목 의 예선 기준 기록 은 1 분 47 초 02 이 다', '1785 년 정조 9 당시 성균관 대사성 민 종현 이 왕명 에 의해 편찬 한 태학 지 건치 에 반궁 도 가 목판화 로 있 는데 본 계첩 과 30 여 년 의 시간 적 차이 가 있 어서 그간 에 변화 된 성균관 의 건물 을 비교 해 볼 수 있 다']
7051665


In [ ]:
'''
# 제너레이터생성
# Python 제너레이터(generator)를 사용하면 실제로 필요할 때까지 Python이 메모리에 아무 것도 로드하지 않도록 할 수 있다.
# 이러한 생성기를 만들려면 꺽쇠괄호(brackets)를 소괄호(parentheses)로 바꾸기만 하면 된다.
def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["whole_func_string"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )

training_corpus = get_training_corpus()
'''

In [4]:
#yield 문을 사용하여 for 루프 내에서 제너레이터(generator)를 정의할 수도 있다.
def get_generator_corpus(max_len: int=100000):
    print(f"len:{max_len}")
    dataset = data
    for start_idx in range(0, len(dataset), max_len):
        samples = dataset[start_idx : start_idx + max_len]
        #print(samples)
        yield samples
        
training_corpus = get_generator_corpus()

In [5]:
training_corpus

<generator object get_generator_corpus at 0x7f4ea1f77660>

In [6]:
# 기존 bert-base-cased 모델 로딩
from transformers import AutoTokenizer
old_tokenizer = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [7]:
# 기존 토큰 테스트 
example = '''i love you
    오늘은 날씨가 참 좋다.
    '''

tokens = old_tokenizer.tokenize(example)
tokens

['i',
 'love',
 'you',
 '오',
 '##늘',
 '##은',
 '날',
 '##씨',
 '##가',
 '참',
 '좋',
 '##다',
 '.']

In [8]:
# 학습 시작 (**오래 걸림)
# => 새로운 토큰을 만듬. vocab 수는 32000개
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 62000)

len:100000





In [9]:
# 새롭게 만든 토큰 테스트 
tokens = tokenizer.tokenize(example)
tokens

['i', 'love', 'you', '오늘', '##은', '날씨', '##가', '참', '좋', '##다', '[UNK]']

In [10]:
# 학습된 tokenizer 저장
tokenizer.save_pretrained("../../data11/my_corpus/vocab/bert-re-kowiki202206-moco2-mecab-62000")

('../../data11/my_corpus/vocab/bert-re-kowiki202206-moco2-mecab-62000/tokenizer_config.json',
 '../../data11/my_corpus/vocab/bert-re-kowiki202206-moco2-mecab-62000/special_tokens_map.json',
 '../../data11/my_corpus/vocab/bert-re-kowiki202206-moco2-mecab-62000/vocab.txt',
 '../../data11/my_corpus/vocab/bert-re-kowiki202206-moco2-mecab-62000/added_tokens.json',
 '../../data11/my_corpus/vocab/bert-re-kowiki202206-moco2-mecab-62000/tokenizer.json')

In [ ]:
'''
# Hugging face 허브에 저장.
# => 로그인 : 허깅페이스(https://huggingface.co)-settings 가면 access token이 있음
from huggingface_hub import notebook_login
notebook_login()

# 노트북 환경이 아닌 경우에는 터미널에서 아래 입력
# huggingface-cli login
'''

In [ ]:
'''
# tokenizer 허깅 페이스 허브에 푸시함.
# 해당 이름으로 리포지토리 생성되고 추가됨
tokenizer.push_to_hub("bert-tokenizer-kowiki20220620-32000", use_temp_dir=True)
'''

In [ ]:
'''
# 추가한 tokenizer 불러옴.
mytokenizer = AutoTokenizer.from_pretrained("kobongsoo/bert-tokenizer-kowiki20220620-32000")
tokens = mytokenizer.tokenize(example)
tokens
'''